Задача проекта - составить таблицу для рекомендательной системы.

Продакт-менеджер школы SkillFactory создаёт рекомендательную систему, которая предлагает пользователю к купленному курсу еще два популярных курса. Для реализации этого нам нужно выгрузить из системы курсы, которые пользователи покупают чаще всего в связке, и выбрать из них самые популярные. 

После выгрузки SQL-запроса, у нас получилась таблица с двумя столбцами: user_id и resource_id. Наша задача - сгруппировать номера курсов по пользователям, чтобы посмотреть, какие пользователи покупали какие курсы, и отранжировать курсы по популярности.

Загрузим полученные из базы SQL данные и изучим их.

In [1]:
import pandas as pd
from collections import defaultdict

# прочитаем выгруженный файл
data = pd.read_csv('final_sql_query.csv')

data.head()

,user_id,resource_id
0,1010882,490
1,1010802,514
2,1011556,490
3,1120298,514
4,750528,552


In [2]:
# посмотрим на типы наших данных
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70581 entries, 0 to 70580
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      70581 non-null  int64
 1   resource_id  70581 non-null  int64
dtypes: int64(2)
memory usage: 1.1 MB


Проверим, есть ли у нас дубликаты (спойлер: есть!)

In [3]:
sum(data.duplicated())

157

Получается, что 157 пользователей купили один и тот же курс два раза. Возможно, и не 157, а меньше, и купили не два раза, а больше, но это сейчас не важно. Избавимся от дубликатов.

In [4]:
data.drop_duplicates(inplace=True)

Теперь сгруппируем данные по пользователям, чтобы увидеть, сколько курсов купил каждый пользователь.

In [5]:
users_by_courses = data.groupby('user_id', as_index=False).count()
users_by_courses.head()

,user_id,resource_id
0,51,2
1,974,1
2,2233,1
3,2581,1
4,3480,1


Оставим только тех, кто купил более одного курса.

In [6]:
users_with_more_courses = users_by_courses[users_by_courses['resource_id'] > 1]
users_with_more_courses.head()

,user_id,resource_id
0,51,2
6,6117,3
12,10275,2
13,10457,2
20,17166,2


Сформируем список.

In [7]:
# список пользователей, которые купили более одного курса
users_with_more_courses_set = set(users_with_more_courses['user_id'])

Теперь нам нужно понять, какие курсы пользователи покупали вместе с другими.

In [8]:
# создадим словарь, в котором ключами будут id пользователей, купивших более одного курса,
# а значениями - списки купленных ими курсов
courses = data.groupby('user_id')['resource_id'].apply(lambda x: list(x)).reset_index()
courses = courses[courses['user_id'].isin(users_with_more_courses_set)] # убираем дубликаты
courses = courses[courses['resource_id'].map(len) > 1] # оставляем только пользователей, которые купили два и более курса
courses_dict = courses.to_dict()

In [9]:
# разобъем все курсы на пары
# т.к. нам понадобится информация о количестве пар, для хранения используем словарь, где
# в качестве ключа будет кортеж из пары курсов, отсортированных по возрастанию, 
# а в качестве значения - количество раз, которое эта пара встречается в покупках.
# для генерации всех пар ключей из списка удобно воспользоваться функией combinations из модуля itertools
import itertools
course_pairs = defaultdict(int)
for course_set in courses_dict['resource_id'].values():
    pairs = itertools.combinations(course_set, 2)
    for pair in pairs:
        tuple_pair = tuple(sorted(pair))
        course_pairs[tuple_pair] += 1

In [10]:
print(len(course_pairs))

3989


У нас получилось 3989 пар курсов в покупках клиентов.

Создадим таблицу с тремя столбцами:<br>
* Столбец 1. Курс, к которому идёт рекомендация
* Столбец 2. Курс для рекомендации № 1 (самый популярный)
* Столбец 3. Курс для рекомендации № 2 (второй по популярности).

In [11]:
# столбец №1 - это уникальный список всех курсов, которые встречаются в наших ключах
# в библиотеке itertools есть функция, которая поможет нам "раскрыть" список
col_1 = list(set(itertools.chain(*course_pairs.keys()))) 
print("Всего получилось {} курсов, для которых можно подобрать рекомендации.".format(str(len(col_1))))

Всего получилось 126 курсов, для которых можно подобрать рекомендации.


In [12]:
# столбцы №2 и №3 - это курсы, которые чаще всего встречается в парах с этим курсом.
col_2 = list()
col_3 = list()
times_2 = list()
times_3 = list()
# вспомогательная функция, возвращает искомый курс из кортежа
def the_other_course(pair, course):
    if pair[0] == course:
        return pair[1]
    else:
        return pair[0]
# запустим цикл по всем выбранным нами курсам    
for course in col_1:
    # отфильтруем курсы: выберем те пары, где встречается course
    cur_course_pairs = dict(filter(lambda item: item[0][0] == course or item[0][1] == course, course_pairs.items()))
    # отсортируем получившийся словарь по убыванию
    sorted_ccp = {key: value for key, value in sorted(cur_course_pairs.items(), key=lambda item: item[1], reverse=True)}
    # запишем вторые курсы из двух самых больших значений в наши колонки
    # и заодно запишем количество раз, которое эти курсы встречаются вместе
    sorted_ccp_iter = iter(sorted_ccp.items())
    pair, times = next(sorted_ccp_iter)
    col_2.append(the_other_course(pair, course))
    times_2.append(times)
    pair, times = next(sorted_ccp_iter)
    col_3.append(the_other_course(pair, course))
    times_3.append(times)

In [13]:
# соберем получившиеся данные в DataFrame
recommendations = pd.DataFrame({'1stCourse': col_1, '2ndCourse': col_2, 'Occurences1&2': times_2, '3rdCourse': col_3, 'Occurences1&3': times_3}, columns=['1stCourse', '2ndCourse', 'Occurences1&2', '3rdCourse', 'Occurences1&3'])

In [14]:
recommendations.head()

,1stCourse,2ndCourse,Occurences1&2,3rdCourse,Occurences1&3
0,513,503,55,551,38
1,514,551,200,515,139
2,515,551,417,489,286
3,516,745,199,553,107
4,517,551,52,750,34


В целом, таблица с рекомендациями готова. Однако есть еще один момент: а что, если рекомендация встречается слишком мало раз? В этом случае логично будет определить минимальную границу и рекомендовать какой-то другой курс. <br>
В качестве минимальной границы установим 10-й процентиль. Это такой показатель, выше которого находятся 90% всех данных.

In [15]:
print("10-й процентиль количества рекомендаций для первого курса: " + str(recommendations['Occurences1&2'].quantile(q=0.1)))
print("10-й процентиль количества рекомендаций для второго курса: " + str(recommendations['Occurences1&3'].quantile(q=0.1)))

10-й процентиль количества рекомендаций для первого курса: 5.0
10-й процентиль количества рекомендаций для второго курса: 3.5


Это значит, что мы берем рекомендацию для первого курса, если количество совпадений - больше 5, а для второго - больше 3.
Если же количество совпадений меньше - будем рекомендовать один из самых популярных курсов, если мы его еще не рекомендовали.

In [16]:
# создадим константы
FIRST_THRESH = 5
SECOND_THRESH = 3

Сначала составим список из 4х самых популярных курсов. Нам хватит 4х, т.к. у нас 3 колонки.

In [17]:
popular = data['resource_id'].value_counts().index[:4].tolist()
print(popular)

[551, 566, 490, 794]


Будем подставлять в нужную колонку тот из столбцов, где количество совпадений - меньше выбранного нами количества.

In [18]:
def to_popular(course1, course2, course3, popular):
    """
    функция to_popular принимает на входе три курса, и выдает первый по очереди популярный курс, который 
    не является ни одним из трех входящих курсов.
    """
    for course in popular:
        if course1 != course and course2 != course and course3 != course:
            return course
    return 0

for _, row in recommendations.iterrows():
    if row['Occurences1&2'] <= FIRST_THRESH:
        row['2ndCourse'] = to_popular(row['1stCourse'], row['2ndCourse'], row['3rdCourse'], popular)
    if row['Occurences1&3'] <= SECOND_THRESH:
        row['3rdCourse'] = to_popular(row['1stCourse'], row['2ndCourse'], row['3rdCourse'], popular)

Сохраним получившиеся колонки в отдельный датафрейм и выгрузим в формат .csv

In [19]:
# выберем колонки, которые будем сохранять в файл
to_export = recommendations[['1stCourse', '2ndCourse', '3rdCourse']].copy()
# отсортируем по первому курсу по возрастанию
to_export.sort_values(by=['1stCourse'], inplace=True)
# сделаем колонку с первым курсом индексом, т.к. у нас в ней только уникальные значения
to_export.set_index(['1stCourse'], inplace=True)
# выгрузим в .csv
to_export.to_csv('ExportedRecommendations.csv')